In [ ]:
# Import packages.
import cv2
import pickle
import os


BASE_DIR = os.getcwd()    # get the path location of this python file (or the path location to this notebook).
image_dir = os.path.join(BASE_DIR, "images")    # get path location of 'images' file.

In [ ]:
# Function to create a new folder if user wants to add more training data.
def create_folder(directory):
    os.chdir(image_dir)    # change directory to be in 'images'.
    if not os.path.exists(directory):    # if directory does not already exists.
        os.makedirs(directory)
    else:
        raise FileExistsError
    os.chdir(BASE_DIR)    # change directory back to THIS python file, so rest of the code can be executed.


# Ask user if new training data wants to be added.
user_input = input("Would you like to add new training data? ")
user_input.lower()    # lower-case the input.

# If user input is 'yes', then create new folder with specified name given by the user.
if user_input == "yes":
    folder_name = input("Name of new training data? ")
    create_folder(folder_name)
    new_folder = os.path.join(image_dir, folder_name)    # get path location of the new folder.
    print()
else:
    pass
    print()

In [ ]:
# Intel built-in haar cascade classifiers. Use for detection.
face_cascade = cv2.CascadeClassifier("cascades/data/haarcascade_frontalface_alt2.xml")    # front-of-face cascade.
eye_cascade = cv2.CascadeClassifier("cascades/data/haarcascade_eye.xml")    # eyes cascade.
eyeglasses_cascade = cv2.CascadeClassifier("cascades/data/haarcascade_eye_tree_eyeglasses.xml")    # eyeglasses cascade.

# Creating an LBPH (Local Binary Patterns Histograms) Face Recognizer then read the training data.
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("training_data.yml")

# First, load the label-ID dictionary from training. Then, invert that dictionary to be {ID : "person's name"}.
with open("labels.pickle", "rb") as f:
    og_labels = pickle.load(f)    # load the label-ID dictionary.
    labels = {v: k for k, v in og_labels.items()}    # inverting the dictionary to be {Value : Key}.

In [ ]:
capture = cv2.VideoCapture(0)    # use the primary designated camera; usually webcam.

img_index = 1    # dummy variable to increment within While loop to save images.
while True:
    # Capture frame-by-frame.
    ret, frame = capture.read()

    # Converting to a gray frame for detection.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)   # detect faces within the gray frame.
    for (x, y, w, h) in faces:
        print(x, y, w, h)    # visualize x,y coordinates and w,h of face
        roi_gray = gray[y:y+h, x:x+w]    # Region Of Interest in gray frame [y_start:y_end, x_start:x_end].
        roi_color = frame[y:y+h, x:x+w]    # Region Of Interest in color frame.

        # Drawing rectangle around ROI, or face.
        color = (0, 255, 0)    # BGR 0-255.
        stroke = 2    # thickness of line.
        x_end = x + w
        y_end = y + h
        cv2.rectangle(frame, (x, y), (x_end, y_end), color, stroke)    # draw on the original color frame.

        # Detect eyes & draw rectangles.
        eyes = eye_cascade.detectMultiScale(roi_gray)    # detect eyes within the gray ROI implemented above.
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (200, 0, 0), 2)    # draw on the ROI_color.

        # Detect eyeglasses.
        eyeglasses = eyeglasses_cascade.detectMultiScale(roi_gray)

        # Predict the ROI (face) in grayscale, and return the ID and confidence.
        # NOTE: the confidence value ('conf') is the distance to the closest item in the database. so, 100 is better
        # than 200, and 0 would be a "perfect match".
        id_, conf = recognizer.predict(roi_gray)
        if conf <= 70:
            print("ID:", id_)
            print("Name:", labels[id_])
            print("Confidence:", conf)
            print()
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color = [50, 50, 240]
            stroke = 2
            cv2.putText(frame, name, (x, y), font, 1, color, stroke, cv2.LINE_AA)

        # If 'img_index' is less than or equal to 50 and 'user_input' is 'yes', then execute the following.
        if img_index <= 50 and user_input == "yes":
            img_item = "%d.png" % img_index    # creating name for the new image to be saved.
            cv2.imwrite(img_item, roi_gray)    # saving image of the ROI (face) in grayscale.
            new_path = "%s/%s" % (os.path.abspath(new_folder), img_item)   # getting the path location of the saved img.
            os.rename(img_item, new_path)    # move the saved image to the new folder created for it. Function
            # parameters are as follows: (prev name, new name).
            img_index += 1

    # Display the resulting frame.
    cv2.imshow("FACE DETECTION", frame)
    # If the 'img_index' becomes greater than 50 (only happens when the user is saving new training data), then end the
    # frame.
    if (cv2.waitKey(20) & 0xFF == ord("q")) or img_index > 50:
        break

In [ ]:
# When everything is done, release the capture.
capture.release()
cv2.destroyAllWindows()